In [1]:
import pandas as pd
import numpy as np

import string
import re

RANDOM = 42

Load the data

In [2]:
haikus_train_df = pd.read_pickle('./data/haikus_train_df.pickle')
haikus_test_df = pd.read_pickle('./data/haikus_test_df.pickle')

## RNN words

In [3]:
from keras.models import Sequential, load_model
from keras.layers import Dense, Input, LSTM, Dropout
from keras.utils import to_categorical
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.
C:\Users\vi_ci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\vi_ci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\vi_ci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\vi_ci\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [11]:
flatten = lambda l: [item for sublist in l for item in sublist]

corpuswords_raw = flatten(list(haikus_train_df['text_withtokens_clean']))

words = sorted(set(corpuswords_raw))
word_to_int = dict((w, i) for i, w in enumerate(words))

n_words = len(corpuswords_raw)
n_vocab_words = len(words)
print("Total Words: ", n_words)
print("Total Vocab: ", n_vocab_words)

Total Words:  447891
Total Vocab:  24046


In [18]:
n_poems = len(haikus_train_df)

n_poems

25128

In [13]:
words

['',
 '<eNd>',
 '<nEXt>',
 'a',
 'aaaa',
 'aah',
 'aback',
 'abandon',
 'abandoned',
 'abandoning',
 'abating',
 'abattoir',
 'abbess',
 'abbey',
 'abbot',
 'abbott',
 'abc',
 'abcs',
 'abduction',
 'abed',
 'abel',
 'abelard',
 'aberration',
 'abhor',
 'abhorred',
 'abide',
 'abilene',
 'abjure',
 'ablaze',
 'able',
 'ablowing',
 'aboard',
 'abode',
 'abodes',
 'abolish',
 'abominable',
 'abord',
 'abortion',
 'abound',
 'about',
 'above',
 'abraham',
 'abramoff',
 'abreast',
 'abriman',
 'abroad',
 'abrupt',
 'abruptly',
 'absalom',
 'abscond',
 'absence',
 'absent',
 'absently',
 'absinthe',
 'absolute',
 'absolutely',
 'absolution',
 'absorb',
 'absorbed',
 'absorbing',
 'absorbs',
 'abstain',
 'abstemious',
 'abstract',
 'abstracted',
 'abstractedlyone',
 'abstraction',
 'absurd',
 'abundance',
 'abundant',
 'abuse',
 'abuses',
 'abydos',
 'abyss',
 'abysses',
 'acacia',
 'academy',
 'acadian',
 'acadians',
 'acc',
 'acceleration',
 'accent',
 'accents',
 'accept',
 'acceptable',


In [22]:
# prepare the dataset of input to output pairs encoded as integers
seq_length = 15

poemX = []
poemY = []
n_patterns = 0

corpusX = []
corpusY = []
for poem_index in range(0, n_poems):

    wordX = []
    wordY = []
    poem = haikus_train_df['text_withtokens_clean'].iloc[poem_index]
    # add padding to poem
    poem = list(np.full(seq_length - 1, '')) + list(poem)
    for i in range(0,  len(poem) - seq_length, 1):
        seq_in = poem[i:i + seq_length]
        seq_out = poem[i + seq_length]
        wordX.append([word_to_int[word] for word in seq_in])
        wordY.append(word_to_int[seq_out])

    n_patterns = max(n_patterns, len(wordX))
    
    poemX.append(wordX)
    poemY.append(wordY)
    
    corpusX += wordX
    corpusY += wordY

n_wordpatterns = len(corpusX)
print("Total Patterns: ", n_wordpatterns)

Total Patterns:  422763


In [15]:
seq_in

['',
 '',
 '',
 '',
 'quaker',
 'meeting',
 '<nEXt>',
 'through',
 'stovepipe',
 'heated',
 'air',
 '<nEXt>',
 'the',
 'world',
 'shimmers']

In [19]:
corpusX

[[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 633],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 633, 13971],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 633, 13971, 2],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872],
 [0, 0, 0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771],
 [0, 0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0],
 [0, 0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0],
 [0, 0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0, 0],
 [0, 0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0, 0, 2],
 [0, 0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0, 0, 2, 254],
 [0, 0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0, 0, 2, 254, 2244],
 [0, 633, 13971, 2, 10474, 20998, 1872, 1771, 0, 0, 0, 2, 254, 2244, 19965],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 609],
 [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 609, 1

In [20]:
len(corpusY)

422763

In [23]:
# reshape X to be [samples, time steps, features]
X = np.reshape(corpusX, (n_wordpatterns, seq_length, 1))
# normalize
X = X / float(n_vocab_words)
# one hot encode the output variable
y = to_categorical(corpusY)

In [25]:
model = Sequential()
model.add(LSTM(256, input_shape=(X.shape[1], X.shape[2]), return_sequences=True, activation='relu'))
model.add(Dropout(0.2))
model.add(LSTM(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [11]:
model = load_model('weights/word/word_weights-cont-168-4.9039.hdf5')

In [26]:
# define the checkpoint
filepath="weights/word/word-weights-new-{epoch:02d}-{loss:.4f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='loss', verbose=1, save_best_only=True, mode='min')
callbacks_list = [checkpoint]

In [ ]:
# fit the model
history = model.fit(X, y, epochs=200, batch_size=128, callbacks=callbacks_list)


Epoch 1/200
422763/422763 [==============================] - 197s 465us/step - loss: 6.3839

Epoch 00001: loss improved from inf to 6.38391, saving model to weights/word/word-weights-new-01-6.3839.hdf5
Epoch 2/200
422763/422763 [==============================] - 230s 544us/step - loss: 6.1840

Epoch 00002: loss improved from 6.38391 to 6.18402, saving model to weights/word/word-weights-new-02-6.1840.hdf5
Epoch 3/200
422763/422763 [==============================] - 226s 534us/step - loss: 6.0743

Epoch 00003: loss improved from 6.18402 to 6.07430, saving model to weights/word/word-weights-new-03-6.0743.hdf5
Epoch 4/200
422763/422763 [==============================] - 240s 568us/step - loss: 5.9969

Epoch 00004: loss improved from 6.07430 to 5.99686, saving model to weights/word/word-weights-new-04-5.9969.hdf5
Epoch 5/200
422763/422763 [==============================] - 210s 496us/step - loss: 5.9360

Epoch 00005: loss improved from 5.99686 to 5.93595, saving model to weights/word/word-